In [1]:
import numpy as np

## Pressure Vessel Optimisation ##

# There are 4 variables that describe a pressure vessel:
# z1 = 0 # Shell thickness
# z2 = 0 # Head thickness
# x3 = 0 # Inner radius
# x4 = 0 # vessel length (excluding head)

# Our input variables:
# x1,x2 = 0,0

# Decision Vector:
# np.array(x1,x2,x3,x4)





# The shell head and thickness can be expressed as
# z1 = 0.0625* x1
# z2 = 0.0625* x2

# Counter variables:
objective_call_count = 0
g1_call_count = 0
g2_call_count = 0
g3_call_count = 0
g4_call_count = 0

# Constraints:
# x1, x2 are integers between 1 and 99 
# x3, x4 are continuous values between 10 and 200




In [2]:
def calculate_zs(xs):
    z1 = (xs[0]*0.0625)
    z2 = (xs[1]*0.0625)
    return z1, z2


In [3]:
def objective(xs, z1, z2):
    objective_call_count += 1
    x3 = xs[2]
    x4 = xs[3]

    xnew = ((1.7781)*(z2)*(x3**2)) + ((0.6224)*(z1)*(x3)*(x4)) + ((3.1661)*(z1**2)*(x4)) + ((19.84)*(z1**2)*(x3))


In [5]:
def g1(xs, z2):
    g1_call_count += 1

    if (xs[2]*0.00954 <= z2):
        return True
    else:
        return False

def g2(xs, z1):
    g2_call_count += 1

    if (xs[2]*0.0193 <= z1):
        return True
    else:
        return False
        
def g3(xs):
    g3_call_count += 1

    if (xs(3) <= 240):
        return True
    else:
        return False

def g4(xs):
    g4_call_count += 1

    if((-1)*(np.pi)*(xs[2]**2)*(xs[3]) - ((4/3)*np.pi)*(xs[2]**3) <= -1296000):
        return True
    else:
        return False

In [6]:
def main():
    xs = np.array([50, 10, 100.5, 100.5])
    z1,z2 = calculate_zs(xs)
    if (g1(xs, z2) == True and g2(xs, z1) == True and g3(xs) == True and g4(xs) == True):
        objective(xs, z1, z2)
    else:
        print(f"Constraints not met!")

In [8]:
main()

UnboundLocalError: local variable 'g1_call_count' referenced before assignment